In [3]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import caffe
import os

caffe.set_device(0)
caffe.set_mode_gpu()
#caffe.set_mode_cpu()

# Define the auxiliary function for the program run:

def load_model_net(model_used):
    # Set the right path to your model definition file, pretrained model weights,
    # and the image you would like to classify.
    MODEL_FILE = '/home/gonzalo/models/VGG_FACE_deploy.prototxt'
    PRETRAINED = '/media/gonzalo/OS/mydata/weights_solvers/'+model_used
    
    # load the model
    caffe.set_mode_gpu()
    caffe.set_device(0)
    net = caffe.Classifier(MODEL_FILE, PRETRAINED,
                           #mean=np.load('/home/gonzalo/data/MIT/mean_files/5_net_mean.npy').mean(1).mean(1),
                           channel_swap=(2,1,0),
                           raw_scale=255,
                           image_dims=(224, 224))
    print "successfully loaded classifier"
    return net

def prediction_net(files, test_path, angle_list, net):
    
    #reset variables
    tp, tp_0, tp_4, tp_8, tp_12, tp_16, tp_20, tp_24 = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
    fp, fp_0, fp_4, fp_8, fp_12, fp_16, fp_20, fp_24 = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
    tpr = []
    # test the performance
    for x in files:
        label = int(x[3])
        angle = x[5:8]
        if angle[0]=='0':
            rot = int(angle[0])
        elif angle[2]== '_':
            rot = int(angle[1])
        else:
            rot = int(angle[1:])
        
        IMAGE_FILE = os.path.join(test_path, x)
        input_image = caffe.io.load_image(IMAGE_FILE)
        # predict takes any number of images,
        # and formats them for the Caffe net automatically
        pred = net.predict([input_image])
        predicted = pred.argmax()
        if predicted==label:
            tp += 1
            if rot == 0:
                tp_0 += 1
            elif rot == 4:
                tp_4 += 1
            elif rot == 8:
                tp_8 += 1
            elif rot == 12:
                tp_12 += 1
            elif rot == 16:
                tp_16 += 1
            elif rot == 20:
                tp_20 += 1
            elif rot == 24:
                tp_24 += 1  
            print predicted, label, 'good'
        else:
            fp += 1
            if rot == 0:
                fp_0 += 1
            elif rot == 4:
                fp_4 += 1
            elif rot == 8:
                fp_8 += 1
            elif rot == 12:
                fp_12 += 1
            elif rot == 16:
                fp_16 += 1
            elif rot == 20:
                fp_20 += 1
            elif rot == 24:
                fp_24 += 1          
            print predicted, label, 'bad'    
    # Measure the TPR for this setup 
    tpr = [tp/(tp+fp), tp_0/(tp_0+fp_0), tp_4/(tp_4+fp_4), tp_8/(tp_8+fp_8), tp_12/(tp_12+fp_12), 
                tp_16/(tp_16+fp_16), tp_20/(tp_20+fp_20), tp_24/(tp_24+fp_24)]
   
    print ' TPR =', tpr[0], ' TPR 0 =', tpr[1], ' TPR 4 =', tpr[2], ' TPR 8 =', tpr[3]
    print ' TPR 12 =', tpr[4], ' TPR 16 =', tpr[5],' TPR 20 =', tpr[6], ' TPR 24 =', tpr[7]
    
    return tpr

def output_global(number_of_images, tpr):
    with open("/home/gonzalo/data/Results/Results.txt", "a") as text_file:
        text_file.write("TPR for "+ str(number_of_images) + " images: {}\n".format(tpr[0]))

def output_angles(number_of_images, angle_list, tpr):
    with open("/home/gonzalo/data/Results/Results_angle.txt", "a") as text_file:
        text_file.write("TPR for "+ str(number_of_images) + " images and rot=" + 
                        str(angle_list[0]) + ": {}\n".format(tpr[1]))
        text_file.write("TPR for "+ str(number_of_images) + " images and rot=" + 
                        str(angle_list[1]) + ": {}\n".format(tpr[2]))
        text_file.write("TPR for "+ str(number_of_images) + " images and rot=" + 
                        str(angle_list[2]) + ": {}\n".format(tpr[3]))
        text_file.write("TPR for "+ str(number_of_images) + " images and rot=" + 
                        str(angle_list[3]) + ": {}\n".format(tpr[4]))
        text_file.write("TPR for "+ str(number_of_images) + " images and rot=" + 
                        str(angle_list[4]) + ": {}\n".format(tpr[5]))
        text_file.write("TPR for "+ str(number_of_images) + " images and rot=" + 
                        str(angle_list[5]) + ": {}\n".format(tpr[6]))
        text_file.write("TPR for "+ str(number_of_images) + " images and rot=" + 
                        str(angle_list[6]) + ": {}\n".format(tpr[7]))


# Here comes the main function of the testing protocol for 10 runs for each amount of samples
# per subject to obtain average results both on global and angular dependent performance.

if __name__ == "__main__":
    
    # Set the list of samples amount to cycle over during tests:
    img_number = [ 7, 35, 70, 150, 200]
    angle_list =[0, 4, 8, 12, 16, 20, 24]
    test_path = '/home/gonzalo/data/MIT/test/test_0'
    files = os.listdir(test_path)
    
    for t in img_number:

        number_of_images = t
        n = img_number.index(t)
        train_set = 'train_0'
        epoch_set = 'top'
        model_used = 'model'+str(number_of_images)+'_'+train_set+'_'+epoch_set+'.caffemodel'
        print model_used
        
        sum_global, sum_0, sum_4, sum_8, sum_12, sum_16, sum_20, sum_24 = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
        avg_tprs=[]
        
        for i in range(10):
            net = load_model_net(model_used)
            print "Begin testing"    
            tpr = prediction_net(files, test_path, angle_list, net)
            
            sum_global += tpr[0]
            sum_0 += tpr[1]
            sum_4 += tpr[2]
            sum_8 += tpr[3]
            sum_12 += tpr[4]
            sum_16 += tpr[5]
            sum_20 += tpr[6]
            sum_24 += tpr[7]
        
        # Append to the list of TPR (global and per each angle)
        avg_tprs.append(sum_global/10)
        avg_tprs.append(sum_0/10)
        avg_tprs.append(sum_4/10)
        avg_tprs.append(sum_8/10)
        avg_tprs.append(sum_12/10)
        avg_tprs.append(sum_16/10)
        avg_tprs.append(sum_20/10)
        avg_tprs.append(sum_24/10)
    
        
        print "writing to output files"
        output_global(number_of_images, avg_tprs)
        output_angles(number_of_images, angle_list, avg_tprs)


model10_train_0_top.caffemodel
successfully loaded classifier
Begin testing
3 3 good
3 3 good
0 9 bad
0 1 bad
0 8 bad
5 5 good
5 5 good
5 4 bad
0 0 good
2 2 good
0 0 good
0 8 bad
0 7 bad
0 1 bad
0 0 good
5 5 good
2 2 good
3 3 good
5 4 bad
1 1 good
0 9 bad
0 4 bad
3 7 bad
8 2 bad
6 6 good
0 1 bad
5 4 bad
3 4 bad
3 9 bad
0 8 bad
0 6 bad
0 0 good
3 3 good
0 7 bad
3 3 good
0 8 bad
6 6 good
0 0 good
2 2 good
9 9 good
0 4 bad
5 4 bad
0 8 bad
0 0 good
9 9 good
3 3 good
8 5 bad
0 8 bad
0 1 bad
8 2 bad
0 0 good
3 3 good
0 8 bad
0 7 bad
5 4 bad
2 2 good
1 1 good
0 9 bad
6 6 good
0 8 bad
3 5 bad
2 2 good
3 5 bad
0 1 bad
1 1 good
4 4 good
6 6 good
0 0 good
5 4 bad
8 8 good
8 1 bad
2 2 good
0 8 bad
0 7 bad
0 8 bad
0 0 good
0 8 bad
5 4 bad
0 0 good
3 7 bad
0 1 bad
6 6 good
8 5 bad
3 3 good
3 3 good
3 7 bad
2 2 good
0 6 bad
3 3 good
0 7 bad
3 3 good
0 0 good
9 9 good
0 8 bad
2 2 good
0 1 bad
3 3 good
3 3 good
3 7 bad
6 6 good
0 7 bad
0 8 bad
0 9 bad
0 6 bad
6 6 good
0 5 bad
5 4 bad
3 4 bad
8 5 bad
0 

1 1 good
0 7 bad
0 7 bad
9 9 good
0 8 bad
0 5 bad
0 9 bad
0 1 bad
2 2 good
1 1 good
2 2 good
1 1 good
3 3 good
0 7 bad
0 8 bad
6 6 good
0 1 bad
0 0 good
0 9 bad
0 7 bad
3 5 bad
0 9 bad
0 0 good
9 9 good
2 2 good
2 2 good
0 7 bad
9 9 good
0 0 good
3 3 good
6 6 good
8 1 bad
0 8 bad
0 7 bad
3 3 good
8 2 bad
3 7 bad
0 7 bad
3 7 bad
0 8 bad
0 7 bad
0 0 good
1 1 good
0 0 good
0 7 bad
5 5 good
6 6 good
5 4 bad
3 4 bad
0 4 bad
3 5 bad
0 3 bad
3 4 bad
0 0 good
6 6 good
0 6 bad
3 3 good
2 2 good
5 5 good
0 7 bad
0 6 bad
3 5 bad
0 8 bad
2 2 good
2 2 good
6 6 good
3 3 good
0 0 good
6 6 good
5 5 good
3 3 good
2 1 bad
6 6 good
0 0 good
6 6 good
3 4 bad
3 4 bad
6 6 good
0 5 bad
3 3 good
0 0 good
3 3 good
0 8 bad
2 2 good
5 5 good
2 2 good
3 5 bad
3 4 bad
0 7 bad
5 5 good
0 5 bad
5 5 good
9 9 good
3 3 good
3 3 good
0 6 bad
9 9 good
0 1 bad
0 0 good
9 9 good
0 0 good
1 1 good
5 4 bad
0 0 good
0 9 bad
1 1 good
0 8 bad
6 6 good
 TPR = 0.484  TPR 0 = 0.45  TPR 4 = 0.4  TPR 8 = 0.466666666667
 TPR 12 = 0.5

1 1 good
0 9 bad
0 4 bad
3 7 bad
8 2 bad
6 6 good
0 1 bad
5 4 bad
3 4 bad
3 9 bad
0 8 bad
0 6 bad
0 0 good
3 3 good
0 7 bad
3 3 good
0 8 bad
6 6 good
0 0 good
2 2 good
9 9 good
0 4 bad
5 4 bad
0 8 bad
0 0 good
9 9 good
3 3 good
8 5 bad
0 8 bad
0 1 bad
8 2 bad
0 0 good
3 3 good
0 8 bad
0 7 bad
5 4 bad
2 2 good
1 1 good
0 9 bad
6 6 good
0 8 bad
3 5 bad
2 2 good
3 5 bad
0 1 bad
1 1 good
4 4 good
6 6 good
0 0 good
5 4 bad
8 8 good
8 1 bad
2 2 good
0 8 bad
0 7 bad
0 8 bad
0 0 good
0 8 bad
5 4 bad
0 0 good
3 7 bad
0 1 bad
6 6 good
8 5 bad
3 3 good
3 3 good
3 7 bad
2 2 good
0 6 bad
3 3 good
0 7 bad
3 3 good
0 0 good
9 9 good
0 8 bad
2 2 good
0 1 bad
3 3 good
3 3 good
3 7 bad
6 6 good
0 7 bad
0 8 bad
0 9 bad
0 6 bad
6 6 good
0 5 bad
5 4 bad
3 4 bad
8 5 bad
0 9 bad
6 6 good
0 8 bad
0 9 bad
9 9 good
0 1 bad
5 4 bad
0 5 bad
8 5 bad
8 9 bad
0 7 bad
3 9 bad
0 9 bad
0 7 bad
0 1 bad
0 9 bad
2 2 good
0 1 bad
3 4 bad
8 2 bad
3 3 good
2 2 good
0 8 bad
2 2 good
3 4 bad
2 2 good
6 6 good
8 8 good
0 8 bad


9 9 good
7 7 good
3 5 bad
0 9 bad
3 0 bad
9 9 good
2 2 good
2 2 good
7 7 good
9 9 good
3 0 bad
3 3 good
6 6 good
0 1 bad
7 8 bad
7 7 good
3 3 good
2 2 good
7 7 good
7 7 good
7 7 good
7 8 bad
7 7 good
3 0 bad
2 1 bad
3 0 bad
7 7 good
8 5 bad
6 6 good
3 4 bad
3 4 bad
6 4 bad
3 5 bad
3 3 good
4 4 good
3 0 bad
6 6 good
6 6 good
3 3 good
2 2 good
3 5 bad
7 7 good
6 6 good
3 5 bad
7 8 bad
2 2 good
2 2 good
6 6 good
3 3 good
3 0 bad
3 6 bad
8 5 bad
3 3 good
0 1 bad
6 6 good
3 0 bad
6 6 good
3 4 bad
6 4 bad
3 6 bad
8 5 bad
0 3 bad
0 0 good
3 3 good
7 8 bad
2 2 good
8 5 bad
2 2 good
3 5 bad
4 4 good
7 7 good
3 5 bad
5 5 good
5 5 good
9 9 good
3 3 good
3 3 good
6 6 good
9 9 good
0 1 bad
3 0 bad
9 9 good
3 0 bad
0 1 bad
3 4 bad
3 0 bad
9 9 good
0 1 bad
4 8 bad
6 6 good
 TPR = 0.588  TPR 0 = 0.683333333333  TPR 4 = 0.65  TPR 8 = 0.6
 TPR 12 = 0.566666666667  TPR 16 = 0.5  TPR 20 = 0.5  TPR 24 = 0.5
successfully loaded classifier
Begin testing
3 3 good
3 3 good
9 9 good
0 1 bad
6 8 bad
5 5 good
3 5

7 7 good
3 3 good
7 8 bad
6 6 good
0 0 good
2 2 good
9 9 good
4 4 good
4 4 good
8 8 good
3 0 bad
9 9 good
3 3 good
8 5 bad
6 8 bad
0 1 bad
2 2 good
3 0 bad
3 3 good
8 8 good
7 7 good
3 4 bad
2 2 good
2 1 bad
9 9 good
3 6 bad
7 8 bad
3 5 bad
2 2 good
3 5 bad
0 1 bad
0 1 bad
3 4 bad
6 6 good
0 0 good
3 4 bad
6 8 bad
0 1 bad
2 2 good
6 8 bad
7 7 good
3 8 bad
3 0 bad
6 8 bad
3 4 bad
3 0 bad
7 7 good
0 1 bad
6 6 good
5 5 good
3 3 good
3 3 good
7 7 good
2 2 good
6 6 good
3 3 good
7 7 good
3 3 good
3 0 bad
7 9 bad
6 8 bad
2 2 good
0 1 bad
3 3 good
3 3 good
7 7 good
6 6 good
7 7 good
3 8 bad
9 9 good
6 6 good
6 6 good
5 5 good
4 4 good
4 4 good
5 5 good
9 9 good
6 6 good
6 8 bad
9 9 good
9 9 good
0 1 bad
6 4 bad
5 5 good
5 5 good
8 9 bad
7 7 good
9 9 good
0 9 bad
7 7 good
0 1 bad
9 9 good
2 2 good
0 1 bad
3 4 bad
2 2 good
3 3 good
2 2 good
3 8 bad
2 2 good
3 4 bad
2 2 good
6 6 good
6 8 bad
8 8 good
0 0 good
0 0 good
3 4 bad
8 5 bad
0 1 bad
7 7 good
7 7 good
9 9 good
8 8 good
8 5 bad
9 9 good
0

3 3 good
2 2 good
7 7 good
7 7 good
7 7 good
7 8 bad
7 7 good
3 0 bad
2 1 bad
3 0 bad
7 7 good
8 5 bad
6 6 good
3 4 bad
3 4 bad
6 4 bad
3 5 bad
3 3 good
4 4 good
3 0 bad
6 6 good
6 6 good
3 3 good
2 2 good
3 5 bad
7 7 good
6 6 good
3 5 bad
7 8 bad
2 2 good
2 2 good
6 6 good
3 3 good
3 0 bad
3 6 bad
8 5 bad
3 3 good
0 1 bad
6 6 good
3 0 bad
6 6 good
3 4 bad
6 4 bad
3 6 bad
8 5 bad
0 3 bad
0 0 good
3 3 good
7 8 bad
2 2 good
8 5 bad
2 2 good
3 5 bad
4 4 good
7 7 good
3 5 bad
5 5 good
5 5 good
9 9 good
3 3 good
3 3 good
6 6 good
9 9 good
0 1 bad
3 0 bad
9 9 good
3 0 bad
0 1 bad
3 4 bad
3 0 bad
9 9 good
0 1 bad
4 8 bad
6 6 good
 TPR = 0.588  TPR 0 = 0.683333333333  TPR 4 = 0.65  TPR 8 = 0.6
 TPR 12 = 0.566666666667  TPR 16 = 0.5  TPR 20 = 0.5  TPR 24 = 0.5
successfully loaded classifier
Begin testing
3 3 good
3 3 good
9 9 good
0 1 bad
6 8 bad
5 5 good
3 5 bad
3 4 bad
3 0 bad
2 2 good
3 0 bad
7 8 bad
7 7 good
1 1 good
3 0 bad
5 5 good
2 2 good
3 3 good
4 4 good
2 1 bad
9 9 good
4 4 good
7 7 

9 9 good
3 3 good
0 5 bad
8 8 good
0 1 bad
2 2 good
0 0 good
3 3 good
8 8 good
7 7 good
3 4 bad
2 2 good
9 1 bad
9 9 good
6 6 good
0 8 bad
0 5 bad
2 2 good
0 5 bad
0 1 bad
0 1 bad
3 4 bad
6 6 good
0 0 good
3 4 bad
7 8 bad
0 1 bad
2 2 good
7 8 bad
7 7 good
3 8 bad
0 0 good
7 8 bad
3 4 bad
0 0 good
7 7 good
0 1 bad
6 6 good
0 5 bad
3 3 good
0 3 bad
7 7 good
2 2 good
6 6 good
9 3 bad
7 7 good
0 3 bad
0 0 good
9 9 good
0 8 bad
2 2 good
0 1 bad
0 3 bad
0 3 bad
7 7 good
6 6 good
7 7 good
0 8 bad
9 9 good
6 6 good
6 6 good
0 5 bad
0 4 bad
9 4 bad
0 5 bad
9 9 good
6 6 good
0 8 bad
9 9 good
9 9 good
0 1 bad
7 4 bad
0 5 bad
9 5 bad
9 9 good
7 7 good
9 9 good
0 9 bad
7 7 good
0 1 bad
9 9 good
2 2 good
0 1 bad
9 4 bad
2 2 good
0 3 bad
2 2 good
0 8 bad
2 2 good
9 4 bad
2 2 good
6 6 good
7 8 bad
0 8 bad
0 0 good
0 0 good
0 4 bad
0 5 bad
0 1 bad
7 7 good
7 7 good
9 9 good
0 8 bad
0 5 bad
9 9 good
0 1 bad
2 2 good
0 1 bad
2 2 good
0 1 bad
3 3 good
7 7 good
0 8 bad
6 6 good
0 1 bad
0 0 good
9 9 good
7 

0 4 bad
9 4 bad
0 5 bad
0 3 bad
0 4 bad
0 0 good
6 6 good
6 6 good
3 3 good
2 2 good
0 5 bad
7 7 good
6 6 good
3 5 bad
0 8 bad
2 2 good
7 2 bad
6 6 good
3 3 good
0 0 good
6 6 good
0 5 bad
0 3 bad
0 1 bad
6 6 good
0 0 good
6 6 good
0 4 bad
9 4 bad
6 6 good
0 5 bad
9 3 bad
0 0 good
0 3 bad
0 8 bad
2 2 good
0 5 bad
2 2 good
0 5 bad
3 4 bad
7 7 good
0 5 bad
0 5 bad
0 5 bad
9 9 good
0 3 bad
0 3 bad
6 6 good
9 9 good
0 1 bad
0 0 good
9 9 good
0 0 good
0 1 bad
0 4 bad
0 0 good
9 9 good
0 1 bad
0 8 bad
6 6 good
 TPR = 0.52  TPR 0 = 0.5  TPR 4 = 0.525  TPR 8 = 0.5
 TPR 12 = 0.5  TPR 16 = 0.533333333333  TPR 20 = 0.533333333333  TPR 24 = 0.566666666667
successfully loaded classifier
Begin testing
0 3 bad
3 3 good
9 9 good
0 1 bad
0 8 bad
0 5 bad
0 5 bad
3 4 bad
0 0 good
2 2 good
0 0 good
7 8 bad
7 7 good
0 1 bad
0 0 good
0 5 bad
2 2 good
0 3 bad
0 4 bad
0 1 bad
9 9 good
9 4 bad
7 7 good
2 2 good
6 6 good
0 1 bad
3 4 bad
9 4 bad
0 9 bad
0 8 bad
6 6 good
0 0 good
0 3 bad
7 7 good
0 3 bad
7 8 bad
6

3 4 bad
6 6 good
0 0 good
3 4 bad
7 8 bad
0 1 bad
2 2 good
7 8 bad
7 7 good
3 8 bad
0 0 good
7 8 bad
3 4 bad
0 0 good
7 7 good
0 1 bad
6 6 good
0 5 bad
3 3 good
0 3 bad
7 7 good
2 2 good
6 6 good
9 3 bad
7 7 good
0 3 bad
0 0 good
9 9 good
0 8 bad
2 2 good
0 1 bad
0 3 bad
0 3 bad
7 7 good
6 6 good
7 7 good
0 8 bad
9 9 good
6 6 good
6 6 good
0 5 bad
0 4 bad
9 4 bad
0 5 bad
9 9 good
6 6 good
0 8 bad
9 9 good
9 9 good
0 1 bad
7 4 bad
0 5 bad
9 5 bad
9 9 good
7 7 good
9 9 good
0 9 bad
7 7 good
0 1 bad
9 9 good
2 2 good
0 1 bad
9 4 bad
2 2 good
0 3 bad
2 2 good
0 8 bad
2 2 good
9 4 bad
2 2 good
6 6 good
7 8 bad
0 8 bad
0 0 good
0 0 good
0 4 bad
0 5 bad
0 1 bad
7 7 good
7 7 good
9 9 good
0 8 bad
0 5 bad
9 9 good
0 1 bad
2 2 good
0 1 bad
2 2 good
0 1 bad
3 3 good
7 7 good
0 8 bad
6 6 good
0 1 bad
0 0 good
9 9 good
7 7 good
0 5 bad
0 9 bad
0 0 good
9 9 good
2 2 good
2 2 good
7 7 good
9 9 good
0 0 good
3 3 good
6 6 good
0 1 bad
7 8 bad
7 7 good
0 3 bad
7 2 bad
7 7 good
7 7 good
7 7 good
7 8 bad
